In [ ]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import datetime as dt
import seaborn as sns
from sklearn.cluster import KMeans

from src import config

In [ ]:
# customers = pd.read_csv(config.RAW_FILE_PATH / 'olist_customers_dataset.csv')
# orders = pd.read_csv(config.RAW_FILE_PATH / 'olist_orders_dataset.csv')
# order_items = pd.read_csv(config.RAW_FILE_PATH / 'olist_order_items_dataset.csv')

# for df in [order_items, customers, orders]: 
#     print(df.info())

# rfm_order_items = order_items.groupby('order_id').agg({'price': sum, 'freight_value': sum, 'shipping_limit_date': max}).reset_index() 

# rfm_orders = orders.merge(rfm_order_items, on='order_id', how='inner')

# rfm = customers.merge(rfm_orders, on='customer_id', how='inner')

# rfm['order_status'].value_counts()

# rfm = rfm.loc[rfm['order_status'] == 'delivered']

# datetime_cols = [
#   'order_purchase_timestamp',
#   'order_approved_at',
#   'order_delivered_carrier_date',
#   'order_delivered_customer_date',
#   'order_estimated_delivery_date',
# ]

# for col in datetime_cols: 
#     rfm[col] = pd.to_datetime(rfm[col])

# rfm.info()

# rfm = rfm.dropna(axis=0)

# max_date = max(rfm['order_purchase_timestamp']) + dt.timedelta(days=1)

# rfm = rfm.groupby('customer_unique_id').agg({
#     'order_purchase_timestamp': lambda x: (max_date - x.max()).days,
#     'customer_id': 'count',
#     'price': sum
# }).reset_index()

# rfm.columns = ['customer_unique_id', 'recency', 'frequency', 'monetary']

# rfm.describe()

# rfm.to_csv(config.INT_FILE_PATH / 'rfm.csv')

In [ ]:
### scoring

In [ ]:
rfm = pd.read_parquet(config.INT_FILE_PATH / "rfm.parquet")

In [ ]:
rfm.describe()

In [ ]:
rfm.columns

In [ ]:
for col in ["recency", "frequency", "monetary"]:
    fig, _ = plt.subplots(figsize=(10,6))
    sns.boxplot(x=rfm[col])
    plt.show()

In [ ]:
def get_cluster_score(data, column, invert=False):
    data = data
    class_score = f"{column}_score"
    X = pd.DataFrame(data[column])
    
    kmeans_kwargs = {
        "n_clusters": 4,
        "init": "random",
        "n_init": 10,
        "max_iter": 300,
        "random_state": 1,
    }

    def sorted_cluster(x, model=None):
        if model == None:
            model = KMeans()
        model = sorted_cluster_centers_(model, x)
        model = sorted_labels_(model, x)
        return model

    def sorted_cluster_centers_(model, x):
        model.fit(x)
        new_centroids = []
        magnitude = []
        for center in model.cluster_centers_:
            magnitude.append(np.sqrt(center.dot(center)))
        idx_argsort = np.argsort(magnitude)
        model.cluster_centers_ = model.cluster_centers_[idx_argsort]
        return model

    def sorted_labels_(sorted_model, x):
        sorted_model.labels_ = sorted_model.predict(x)
        return sorted_model


    km = KMeans(**kmeans_kwargs).fit(X)
    cluster = sorted_cluster(X, km)
    X[class_score] = cluster.predict(X)


    plt.figure(figsize=(15,3))
    plt.title(f'Results of clustering {column}')

    sns.scatterplot(x=X[column], y=X.index, hue=X[class_score], s=10,)
    
    # append back to original dataset
    data[class_score] = X[class_score] + 1

    if invert:
        data[class_score] = data[class_score].map({4:1, 3:2, 2:3, 1:4})


In [ ]:
rfm.info()

In [ ]:
# make scores 
get_cluster_score(rfm, "recency", invert=True)
get_cluster_score(rfm, "monetary")
get_cluster_score(rfm, "frequency")

In [ ]:
rfm["total_score"] = rfm["recency_score"] + rfm["monetary_score"] + rfm["frequency_score"] 

In [ ]:
rfm["segment_value"] = pd.cut(rfm["total_score"], bins=3, labels=["Low", "Medium", "High"])

In [ ]:
rfm.head()